Reload modules every time before executing the Python code typed:

In [2]:
%load_ext autoreload
%autoreload 2

Install necessary packages:

In [3]:
%pip install opencv-python numpy scipy filterpy==1.4.5

Note: you may need to restart the kernel to use updated packages.


# Task 1 (exercise 1.2)

Import necessary packages:

In [4]:
import cv2
import numpy as np
from scipy.stats import linregress

import sys
sys.path.append('./')

from sort import Sort

The higher the value, the bigger a moving object should be to be detected

In [5]:
MIN_CONTOUR_AREA = 4000

In [15]:
def calc_centroid(contour):
    """
    Calculates the centroid of a contour.
    """
    M = cv2.moments(contour)
    if M["m00"] == 0:
        # Avoid division by zero
        return None

    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    return (cX, cY)

In [23]:
class Tracker:
    class TrackingObject:
        pass

    def __init__(self) -> None:
        self.objects = []

    def update(self, contours, centroids):
        centroids = centroids.copy()
        """
        Updates the tracker with a new frame information: contours and centroids.
        """
        
        

In [33]:
# Open the sample video file:
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Ensure that the file was opened correctly:
assert video.isOpened(), "Can't open the video file"

# Initialize an instance of [Gaussian Mixture-based Background/Foreground Segmentation Algorithm](https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html):
bg_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows=False, history=10000, dist2Threshold=800)

# Instantiate the tracker
tracker = Tracker()

cv2.namedWindow("Cam")

counted_car_ids = set()

# Read the video frame by frame:
while True:
    ret, frame = video.read()

    # Check for the last frame
    if not ret:
        break

    # blur the frame to remove noise
    blured_frame = cv2.GaussianBlur(frame, (5,5), 0)

    # Apply the background subtraction algorithm:
    fg_mask = bg_subtractor.apply(blured_frame)

    # Apply morphological operations to remove corruptions:
    _, thresh = cv2.threshold(fg_mask, 25, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find the contours of the detected objects:
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out the contours that are too small to be a car:
    contours = [c for c in contours if cv2.contourArea(c) > MIN_CONTOUR_AREA]

    # Convex hull to get clean contours without holes:
    contours = [cv2.convexHull(c) for c in contours]

    # Draw the contours on the original image:
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

    # Calculate the centroids of the contours: 
    centroids = []# [calc_centroid(c) for c in contours if c is not None]

    # Draw the centroids on the original image for debugging purposes:
    for centroid in centroids:
        cv2.circle(frame, centroid, 5, (0, 0, 255), -1)

    # Update the tracker with contours and centroids:
    # tracker.update(contours, centroids)

    # Display the resulting frame
    cv2.imshow("Cam", frame)

    # Press "q" to exit the loop
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        cv2.destroyWindow("Cam")
        cv2.waitKey(1)
        break